In [51]:
from langgraph.graph import StateGraph,START, END
from langchain_groq import ChatGroq
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

import os
from dotenv import load_dotenv
load_dotenv()

True

In [52]:
def sanitize_ascii(s: str) -> str:
    # Remove any non-ASCII characters from the string
    return ''.join(c for c in s if ord(c) < 128)

In [53]:
api_key = os.environ["GROQ_API_KEY"]
# api_key = os.environ["GEMINI_API_KEY"]
# api_key = os.environ["MISTRAL_API_KEY"]
# api_key = os.environ["OPENAI_API_KEY"]

model = "llama3-70b-8192"
# model = "gemini-2.0-flash"
# model = "codestral-latest"
# model = "gpt-4.1-nano"

sanitized_api_key = sanitize_ascii(api_key)
llm = ChatGroq(api_key=sanitized_api_key,model=model)
# llm = ChatGoogleGenerativeAI(api_key=api_key,model=model)
# llm = ChatMistralAI(api_key=api_key,model=model)
# llm = ChatOpenAI(api_key=api_key,model=model)



In [54]:
result=llm.invoke("Hello")
result

AIMessage(content="Hello! It's nice to meet you. Is there something I can help you with, or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 11, 'total_tokens': 37, 'completion_time': 0.074285714, 'prompt_time': 0.000118899, 'queue_time': 0.09523881299999999, 'total_time': 0.074404613}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_dd4ae1c591', 'finish_reason': 'stop', 'logprobs': None}, id='run-8b79e953-9275-417c-b20a-9f0d8d1e4d8c-0', usage_metadata={'input_tokens': 11, 'output_tokens': 26, 'total_tokens': 37})

In [55]:
from pydantic import BaseModel, Field
from typing import Any, Optional, List, Dict

class TerraformFile(BaseModel):
    path: str
    content: str

class TerraformState(BaseModel):
    """State for our Terraform code generation agent."""
    files: List[TerraformFile] = Field(default_factory=list)
    requirements: Dict[str, str] = Field(default_factory=dict)
    modules_needed: List[str] = Field(default_factory=list)
    environments: List[str] = Field(default=[])
    current_request: Optional[str] = None


In [65]:
# Define the template
terraform_template = """
You are a Terraform expert. Generate Terraform code based on the user's requirements.
The code should be organized in the following directory structure:

src/
  ├── environments/
  │   ├── dev/
  │   │   ├── main.tf
  │   │   ├── output.tf
  │   │   └── variables.tf
  │   ├── stage/
  │   │   ├── main.tf
  │   │   ├── output.tf
  │   │   └── variables.tf
  │   └── prod/
  │       ├── main.tf
  │       ├── output.tf
  │       └── variables.tf
  └── modules/
      ├── vpc-module/
      │   ├── main.tf
      │   ├── output.tf
      │   └── variables.tf
      └── alb-module/
          ├── main.tf
          ├── output.tf
          └── variables.tf

For each file, provide the complete content wrapped in ```terraform triple backticks.
Return the output as a JSON object with file paths as keys and content as values, for example:

{{
  "src/environments/dev/main.tf": "terraform code here",
  "src/environments/dev/output.tf": "terraform code here",
  ...and so on for all files
}}

User requirements: {requirements}
"""

In [70]:
import json

# Define our graph functions
def process_request(state: TerraformState):
    """Process the user's request and update the state."""
    return state

def generate_terraform_code(state: TerraformState):
    """Generate Terraform code based on requirements in state."""
    # Create prompt
    prompt = PromptTemplate.from_template(terraform_template)
    
    # Get response from LLM
    chain = prompt | llm
    response = chain.invoke({"requirements": state.current_request})
    
    # Parse the response and update state
    # try:
    #     # Extract JSON from response
    #     import re
    #     json_match = re.search(r'```json\n(.*?)\n```', response.content, re.DOTALL)
    #     if json_match:
    #         json_str = json_match.group(1)
    #     else:
    #         json_str = response.content
            
    #     files_dict = json.loads(json_str)
    #     for path, content in files_dict.items():
    #         # Clean up content if it contains markdown code blocks
    #         clean_content = content
    #         if "```terraform" in content:
    #             clean_content = re.search(r'```terraform\n(.*?)\n```', content, re.DOTALL)
    #             if clean_content:
    #                 clean_content = clean_content.group(1)
    #             else:
    #                 clean_content = content
            
    #         state.files.append(TerraformFile(path=path, content=clean_content))
    # except Exception as e:
    #     # Handle parsing errors
    #     print(f"Error parsing LLM response: {e}")
    
    print(response.content)
    return state

# Function to save files to disk
def save_terraform_files(state: TerraformState, base_dir: str = ".") -> None:
    """Save the generated Terraform files to disk."""
    for file in state.files:
        # Create directory if it doesn't exist
        dir_path = os.path.dirname(os.path.join(base_dir, file.path))
        os.makedirs(dir_path, exist_ok=True)
        
        # Write content to file
        with open(os.path.join(base_dir, file.path), "w") as f:
            f.write(file.content)
    
    print(f"Terraform files have been saved to {base_dir}")

# Define the graph
graph = StateGraph(TerraformState)

# Add nodes
graph.add_node("process_request", process_request)
graph.add_node("generate_terraform_code", generate_terraform_code)


# Add edges
graph.add_edge(START, "process_request")
graph.add_edge("process_request", "generate_terraform_code")
graph.add_edge("generate_terraform_code", END)

# Compile the graph
terraform_app = graph.compile()

In [58]:
from langchain_core.runnables.graph import MermaidDrawMethod

img_data = terraform_app.get_graph().draw_mermaid_png(
            draw_method=MermaidDrawMethod.API
            )

# Save the image to a file
graph_path = "workflow_graph.png"
with open(graph_path, "wb") as f:
    f.write(img_data) 

In [71]:
result = terraform_app.invoke({"current_request": "Create a VPC with public and private subnets in AWS with appropriate security groups"})

print(result)

# Save the files to disk
# save_terraform_files(result, "output")

Here is the Terraform code organized in the specified directory structure:

```
{
  "src/environments/dev/variables.tf": ```
variable "aws_region" {
  type        = string
  default     = "us-west-2"
}

variable "vpc_cidr" {
  type        = string
  default     = "10.0.0.0/16"
}

variable "public_subnet_cidr" {
  type        = string
  default     = "10.0.1.0/24"
}

variable "private_subnet_cidr" {
  type        = string
  default     = "10.0.2.0/24"
}
```

  "src/environments/dev/main.tf": ```
provider "aws" {
  region = var.aws_region
}

module "vpc" {
  source = "../../modules/vpc-module"

  vpc_cidr = var.vpc_cidr
  public_subnet_cidr = var.public_subnet_cidr
  private_subnet_cidr = var.private_subnet_cidr
}
```

  "src/environments/dev/output.tf": ```
output "vpc_id" {
  value = module.vpc.vpc_id
}

output "public_subnet_id" {
  value = module.vpc.public_subnet_id
}

output "private_subnet_id" {
  value = module.vpc.private_subnet_id
}
```

  "src/modules/vpc-module/variables.tf":